In [ ]:
import itertools
import numpy as np
import scipy.linalg
import math

def skew(vector):
    """
    Returns the 3x3 skew matrix of the input vector. The skew matrix is a square matrix R whose transpose 
    is also its negative, that is, it satisfies the condition -R = R^T

    Args:
        vector (array_like): The input vector

    Returns:
        array_like: The resulting 3x3 skew matrix
    """
    skv = np.roll(np.roll(np.diag(np.asarray(vector).flatten()), 1, 1), -1, 0)
    return (skv - skv.T)

def quaternion_conjugate(q):
    """
    Compute the conjugate of a quaternion.

    Args:
        q (array_like): Input quaternion (4 element sequence)

    Returns:
        array_like: The conjugate of the input quaternion

    Notes:
    Examples
    --------
    >>> import baldor as br
    >>> q0 = np.array([0.7071, 0, 0, 0.7071])
    >>> q1 = quaternion_conjugate(q0)
    >>> q1[0] == q0[0] and all(q1[1:] == -q0[1:])
    True
    """
    qconj = np.array(q, dtype=np.float64, copy=True)
    np.negative(qconj[1:], qconj[1:])
    return qconj

def quaternion_multiply(q1, q2):
    """
    Multiply two quaternions

    Args:
        q1 (array_like): First input quaternion (4 element sequence)
        q2 (array_like): Second input quaternion (4 element sequence)

    Returns:
        array_like: resulting quaternion after first * second

    Notes:
    See hamilton product in https://en.wikipedia.org/wiki/Quaternion
    Examples
    --------
    >>> import numpy as np
    >>> q = quaternion_multiply([4, 1, -2, 3], [8, -5, 6, 7])
    >>> np.allclose(q, [28, -44, -14, 48])
    """

    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([-x1*x2 - y1*y2 - z1*z2 + w1*w2,
                     x1*w2 + y1*z2 - z1*y2 + w1*x2,
                     -x1*z2 + y1*w2 + z1*x2 + w1*y2,
                     x1*y2 - y1*x2 + z1*w2 + w1*z2], dtype=np.float64)

def quaternion_to_transform(quaternion):
    """
    Return homogeneous transformation from a quaternion. Quaternions w + ix + jy + kz are represented as [w, x, y, z]

    Args:
        quaternion (array_like): Input quaternion (4 element sequence)
    
    Returns:
        array_like: Homogeneous transformation (4x4)

    Notes:
    Examples
    --------
    >>> import numpy as np
    >>> T0 = quaternion_to_transform([1, 0, 0, 0]) # Identity quaternion
    >>> np.allclose(T0, np.eye(4))
    True
    >>> T1 = quaternion_to_transform([0, 1, 0, 0]) # 180 degree rot around X
    >>> np.allclose(T1, np.diag([1, -1, -1, 1]))
    True
    """
    q = np.array(quaternion, dtype=np.float64, copy=True)
    n = np.dot(q, q)
    # if the quaternions are not well formed, use identity
    if n < (np.finfo(np.float64).eps * 4.0):
        return np.identity(4)
    # compute the product and square terms
    q *= math.sqrt(2.0 / n)
    q = np.outer(q, q)
    # https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation
    return np.array([
        [1.0-q[2, 2]-q[3, 3],       q[1, 2]-q[3, 0],        q[1, 3]+q[2, 0],        0.0],
        [q[1, 2]+q[3, 0],           1.0-q[1, 1]-q[3, 3],    q[2, 3]-q[1, 0],        0.0],
        [q[1, 3]-q[2, 0],           q[2, 3]+q[1, 0],        1.0-q[1, 1]-q[2, 2],    0.0],
        [0.0,                       0.0,                    0.0,                    1.0]])

def dual_to_transform(qr, qt):
    """
    Dual quaternion to homogeneous transformation

    Args:
        qr (array_like): Input quaternion for the rotation component (4 element sequence)
        qt (array_like): Input quaternion for the translation component (4 element sequence)

    Returns:
        array_like: Transformation matrix
    """
    T = np.eye(4)
    R = quaternion_to_transform(qr)[:3, :3]
    t = 2*quaternion_multiply(qt, quaternion_conjugate(qr))
    T[:3, :3] = R
    T[:3, 3] = t[1:]
    return T

def transform_to_axis_angle(transform):
    """
    Return rotation angle and axis from rotation matrix

    Args:
        transform (array_like): The input homogeneous transformation

    Returns:
        array_like: axis and angle of rotation, as well as the point at which the rotation is performed. The point specifies where the axis is
    """
    R = np.array(transform, dtype=np.float64, copy=False)
    R33 = R[:3, :3]
    # direction: unit eigenvector of R33 corresponding to eigenvalue of 1
    w, W = np.linalg.eig(R33.T)
    i = np.where(abs(np.real(w) - 1.0) < 1e-8)[0]
    if not len(i):
        raise ValueError("no unit eigenvector corresponding to eigenvalue 1")
    axis = np.real(W[:, i[-1]]).squeeze()
    # point: unit eigenvector of R corresponding to eigenvalue of 1
    w, Q = np.linalg.eig(R)
    i = np.where(abs(np.real(w) - 1.0) < 1e-8)[0]
    if not len(i):
        raise ValueError("no unit eigenvector corresponding to eigenvalue 1")
    point = np.real(Q[:, i[-1]]).squeeze()
    point /= point[3]
    # rotation angle depending on axis
    cosa = (np.trace(R33) - 1.0) / 2.0
    if abs(axis[2]) > 1e-8:
        sina = (R[1, 0] + (cosa-1.0)*axis[0]*axis[1]) / axis[2]
    elif abs(axis[1]) > 1e-8:
        sina = (R[0, 2] + (cosa-1.0)*axis[0]*axis[2]) / axis[1]
    else:
        sina = (R[2, 1] + (cosa-1.0)*axis[1]*axis[2]) / axis[0]
    angle = math.atan2(sina, cosa)
    return axis, angle, point

def to_dual_quaternion(transform):
    """
    Returns the dual quaternion representation from an SE3 transformation matrix

    Args:
        transform (array_like): SE3 transformation matrix

    Returns:
        array_like: qr and qt for the rotation and translation components respectively
    """
    def cot(x): return 1./np.tan(x)
    R = np.eye(4)
    R[:3, :3] = transform[:3, :3]
    l, theta, _ = transform_to_axis_angle(R)
    t = transform[:3, 3]
    # Pitch d
    d = np.dot(l.reshape(1, 3), t.reshape(3, 1))
    # Point c
    c = 0.5*(t-d*l) + cot(theta/2.)*np.cross(l, t)
    # Moment vector
    m = np.cross(c, l)
    # Rotation quaternion
    qr = np.zeros(4)
    qr[0] = np.cos(theta/2.)
    qr[1:] = np.sin(theta/2.)*l
    # Translation quaternion
    qt = np.zeros(4)
    qt[0] = -(1/2.)*np.dot(qr[1:], t)
    qt[1:] = (1/2.)*(qr[0]*t + np.cross(t, qr[1:]))
    return qr, qt

In [ ]:
class SolverBase(object):
    """
    Base class for the solvers. It encodes the requirements and structure of
    Hand-Eye calibration solvers.
    """

    def __repr__(self):
        return self.__class__.__name__

    def __str__(self):
        return self.__repr__()

    def __call__(self, A, B):
        """
        A solver class must implement this method so that the class can be
        `callable`.

        Args:
            A (list): List of homogeneous transformations with the relative motion of the end-effector
            B (list): List of homogeneous transformations with the relative motion of the calibration pattern (often called `object`)

        Raises:
            array_like: Xhat, the estimate of the homogeneous transformation
            NotImplementedError: _description_
        """
        raise NotImplementedError('A solver must be callable')

    @staticmethod
    def estimate_translation(A, B, Rx):
        """
        Estimate the translation component of Xhat in AX=XB. This
        requires the estimation of the rotation component Rx
        Parameters

        Args:
            A (list): List of homogeneous transformations with the relative motion of the end-effector
            B (list): List of homogeneous transformations with the relative motion of the calibration pattern (often called `object`)
            Rx (array_like): Estimate of the rotation component (rotation matrix) of Xhat

        Returns:
            array_like: The estimated translation component (XYZ value) of Xhat
        """
        C = []
        d = []
        for Ai, Bi in zip(A, B):
            ta = Ai[:3, 3]
            tb = Bi[:3, 3]
            C.append(Ai[:3, :3]-np.eye(3))
            d.append(np.dot(Rx, tb)-ta)
        C = np.array(C)
        C.shape = (-1, 3)
        d = np.array(d).flatten()
        tx, residuals, rank, s = np.linalg.lstsq(C, d, rcond=-1)
        return tx.flatten()

In [5]:
import itertools
import numpy as np
import scipy.linalg

A = [np.random.rand(3,3), np.random.rand(3,3), np.random.rand(3,3)]
B = [np.random.rand(3,3), np.random.rand(3,3), np.random.rand(3,3)]

# print(A[1])
# # print(B)

# print(A[1][:2,2])

# print(A[1][2:,2])

# print(A)

# C = []
# C.append(np.random.rand(3,3))
# C.append(np.random.rand(3,3))

# print(C)

# Cc1 = np.array(C)
# Cc2 = Cc1

# Cc1.shape = (-1,3)
# print(Cc1.shape)
# print(Cc1)


# Cc2.reshape(-1,3)
# print(Cc2.shape)
# print(Cc2)


# print(Cc2.flatten().shape)

w, W = np.linalg.eig(np.random.rand(3,3))

print(w)

i = np.where(abs(np.real(w) > 0))
print(i)

print(W)

[ 1.9169849  -0.41710755  0.12880605]
(array([0, 2]),)
[[-0.55192871 -0.75367648  0.28704712]
 [-0.49392372 -0.02252132 -0.80609046]
 [-0.67187354  0.65685961  0.51751534]]
